In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns

In [73]:
print_to_excel = True

In [74]:
files_list = [i for i in os.listdir() if "~" not in i]
excel_files = [i for i in files_list if '.xlsx' in i and ' ' in i]

sheets_dict = {}
for excel_file in excel_files:
    data = pd.read_excel(excel_file, None)
    sheets_dict[excel_file] = [i for i in data.keys() if 'DE PARA' in i][0]
sheets_dict

{'BRFHRG FX Exposure 24.05.2024.xlsx': 'DE PARA FH',
 'BRFTO FX Exposure Model 24.05.2024.xlsx': 'DE PARA FTO',
 'BRSFT FX Exposure Model 24.05.2024.xlsx': 'DE PARA SALITRE',
 'Cópia de BRFTO FX Exposure Analysis as of 20.03.24 MT comments V2.xlsx': 'DE PARA FTO',
 'Cópia de DE PARA FH Commented 07.05.24 21.05.xlsx': 'DE PARA FH'}

In [90]:
files_dict = {'BRFTO FX Exposure Model 24.05.2024.xlsx':'Cópia de BRFTO FX Exposure Analysis as of 20.03.24 MT comments V2.xlsx', 'BRFHRG FX Exposure 24.05.2024.xlsx':'Cópia de DE PARA FH Commented 07.05.24 21.05.xlsx'}
# loop for files
for file_calculated in list(files_dict.keys()):
    data_reference_work=0
    data_calculated_work=0
    # read data
    data_calculated = pd.read_excel(file_calculated, sheets_dict[file_calculated])
    data_reference = pd.read_excel(files_dict[file_calculated], sheets_dict[files_dict[file_calculated]])
    # work data
    data_reference_work = data_reference.loc[:,['Seg Conta',data_reference.columns[data_reference.columns.str.contains('MOEDA ')].tolist()[0]]]
    data_calculated_work = data_calculated.loc[:,['Seg Conta',data_calculated.columns[data_calculated.columns.str.contains('MOEDA ')].tolist()[0]]]
    # merge
    data_merge = data_reference_work.merge(data_calculated_work, how='outer', left_on='Seg Conta', right_on='Seg Conta').rename(columns={'MOEDA  Exposição_x':'reference', 'MOEDA  Exposição_y':'calculated'})
    data_merge['Cols_is_equal'] = data_merge.iloc[:,1] == data_merge.iloc[:,2]
    data_not_equal = data_merge[data_merge.Cols_is_equal==False]
    if 'BRFTO' in file_calculated:
        data_not_equal_BRFTO = data_not_equal
        data_reference_BRFTO = data_reference
    elif 'BRFHRG' in file_calculated:
        data_not_equal_BRFHRG = data_not_equal
        data_reference_BRFHRG = data_reference
    # to excel
    if print_to_excel == True:
        Output_file=f'{str(date.today())}_Brazil_compaire_mapping.xlsx'
        data_not_equal_BRFTO.to_excel(Output_file, 'BRFTO', index=False)
        new_list(data_not_equal_BRFHRG, Output_file, 'BRFHRG')


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18800\1347511787.py:25: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  data_not_equal_BRFTO.to_excel(Output_file, 'BRFTO', index=False)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18800\1347511787.py:25: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  data_not_equal_BRFTO.to_excel(Output_file, 'BRFTO', index=False)


In [109]:
data_reference_BRFHRG_BRFTO = pd.concat([data_reference_BRFHRG,data_reference_BRFTO])
data_reference_work_BRFHRG_BRFTO = data_reference_BRFHRG_BRFTO[['Seg Conta','Descr Conta3',data_calculated.columns[data_calculated.columns.str.contains('MOEDA ')].tolist()[0]]]


BRSFT_file = [i for i in excel_files if 'BRSFT' in i][0]
data_calculated = pd.read_excel(BRSFT_file, sheets_dict[BRSFT_file])
data_calculated_work = data_calculated.loc[:,['Seg Conta','Descr Conta3',data_calculated.columns[data_calculated.columns.str.contains('MOEDA ')].tolist()[0]]]
# merge
# data_merge = 
data_calculated_work.merge(data_reference_work_BRFHRG_BRFTO, how='left', left_on='Descr Conta3', right_on='Descr Conta3')
# data_merge['Cols_is_equal'] = data_merge.iloc[:,1] == data_merge.iloc[:,2]
# data_not_equal = data_merge[data_merge.Cols_is_equal==False]
# data_not_equal
[i for i in data_calculated_work['Descr Conta3'].tolist() if i in data_reference_work_BRFHRG_BRFTO['Descr Conta3'].tolist()]
# data_reference_work_BRFHRG_BRFTO['Descr Conta3'].tolist()


[]